In [8]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from PIL import Image
from scipy import ndimage
import os
from sklearn.preprocessing import LabelEncoder

In [9]:
base_path = "./cat-bunny/test-images/"

def load_category(base_path, cat):
    X = []
    y = []
    path = os.path.join(base_path, cat)

    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)

        try:
            img = Image.open(img_path).convert("RGB")
            img = img.resize((64, 64))

            X.append(np.array(img))

            if cat == "cat":
                y.append(1)

            else:
                y.append(0)

        except Exception as err:
            print(f"[Error] Fallo en la carga. {err}")
    
    return np.array(X), np.array(y)

X_test = []; y_test = []

for cat in ["cat", "rabbit"]:
    base_path = "./cat-bunny/test-images/"
    X, y = load_category(base_path, cat)

    X_test.append(X); y_test.append(y)

X_test = np.vstack([X_test[0], X_test[1]])
y_test = np.hstack([y_test[0], y_test[1]])

print(X_test.shape)
print(np.array(y_test).shape)

(15, 64, 64, 3)
(15,)


In [10]:
np.unique(y_test)

array([0, 1])

In [11]:
X_train = []; y_train = []

for cat in ["cat", "rabbit"]:
    base_path = "./cat-bunny/test-images/"
    X, y = load_category(base_path, cat)

    X_train.append(X); y_train.append(y)

X_train = np.vstack([X_train[0], X_train[1]])
y_train = np.hstack([y_train[0], y_train[1]])

print(X_train.shape)
print(np.array(y_train).shape)

(15, 64, 64, 3)
(15,)


In [13]:
etiquetas = {0:"conejo",
             1:"gato"}

In [ ]:
index = np.random.randint()